# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [1]:
# You are allowed to change version of transformers or use other toolkits
#!pip install transformers==4.5.0

## Import Packages

In [2]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import optimization

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(0)

In [3]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [4]:
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext-large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [7]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 400
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 80

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #mid = (answer_start_token + answer_end_token) // 2
            mid = np.random.randint(low = answer_start_token-1, high = answer_end_token)
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [8]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        if start_index > end_index:
            prob = 0
        elif end_index - start_index >= 7:
            prob = 0
        else:
            prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            entire_start_index = start_index.item() + doc_stride * k
            entire_end_index = end_index.item() + doc_stride * k
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            answer = answer.replace(' ','')

    if '[UNK]' in answer:
        print('find [UNK] in prediction', answer)
       
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer

## Training

In [9]:
num_epoch = 5
validation = True
logging_step = 200
learning_rate = 1e-4
num_training_steps = len(train_loader)*num_epoch/train_batch_size
accumulation_steps = 32



optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = optimization.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)
if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):
        # Load all data into GPU
        
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss/accumulation_steps
        loss = output.loss/accumulation_steps
        
        
        
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            loss.backward()
        
        if step % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        step += 1    

        ##### TODO: Apply linear learning rate decay #####
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0
    

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 1 | Step 200 | loss = 0.193, acc = 0.000
Epoch 1 | Step 400 | loss = 0.193, acc = 0.000
Epoch 1 | Step 600 | loss = 0.190, acc = 0.000
Epoch 1 | Step 800 | loss = 0.185, acc = 0.000
Epoch 1 | Step 1000 | loss = 0.178, acc = 0.002
Epoch 1 | Step 1200 | loss = 0.170, acc = 0.007
Epoch 1 | Step 1400 | loss = 0.159, acc = 0.021
Epoch 1 | Step 1600 | loss = 0.146, acc = 0.025
Epoch 1 | Step 1800 | loss = 0.131, acc = 0.026
Epoch 1 | Step 2000 | loss = 0.113, acc = 0.065
Epoch 1 | Step 2200 | loss = 0.074, acc = 0.278
Epoch 1 | Step 2400 | loss = 0.053, acc = 0.427
Epoch 1 | Step 2600 | loss = 0.040, acc = 0.536
Epoch 1 | Step 2800 | loss = 0.037, acc = 0.569
Epoch 1 | Step 3000 | loss = 0.034, acc = 0.616
Epoch 1 | Step 3200 | loss = 0.032, acc = 0.638
Epoch 1 | Step 3400 | loss = 0.030, acc = 0.649
Epoch 1 | Step 3600 | loss = 0.028, acc = 0.666
Epoch 1 | Step 3800 | loss = 0.028, acc = 0.659
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

find [UNK] in prediction 李[UNK]
final prediction 農
find [UNK] in prediction 朱允[UNK]
final prediction 昌
find [UNK] in prediction [UNK]崎八幡宮
final prediction 內
find [UNK] in prediction 與慕容[UNK]雙方不和
final prediction 渾
find [UNK] in prediction 1[UNK]0
final prediction 少傳統勁旅失色，衛冕冠軍西班牙僅以小組次名出線淘汰賽，最後在十六強賽以 0–2 敗予義大利，宣告衛冕失敗。同樣僅以小組次名出線的英格蘭，亦在十六強賽以 1–2 爆冷不敵首次晉級歐國盃的冰島。反而一些過去被忽視的小國球隊今屆表現出色，除了淘汰英格蘭首次晉級八強的冰島外，威爾斯不但壓倒英格蘭取得小組首名出線，接下來更淘汰北愛爾蘭及比利時，歷史性晉級歐國盃4強。最後晉級決賽的是主辦國法國及2004年歐國盃亞軍葡萄牙。法國除分組賽與瑞士戰平外，其餘5場比賽皆在法定時間擊敗對手，其中在準決賽以 2–0 擊敗奪標熱門德國，相反葡萄牙分組賽三戰皆和，僅以較佳第三名晉級，但晉級後僥倖避免與多支強隊對賽，在從未在法定時間取得一場勝仗下晉級4強，再以 2–0 淘汰威爾斯入決賽，故賽前法國被看高一線。最終，葡萄牙在主力基斯坦奴·朗拿度比賽早段傷出下，仍與法國在法定時間打成 0–0，更在延長賽下半場葡萄牙後備前鋒艾達射入奠定勝局一球，以 1–0 擊敗法國，首次奪得歐國盃冠軍。
find [UNK] in prediction 杜恆-[UNK]因論題
final prediction 實
find [UNK] in prediction 青翁三足[UNK]
final prediction 博
find [UNK] in prediction [UNK][UNK]
final prediction 
find [UNK] in prediction 朱允[UNK]
final prediction 於
find [UNK] in prediction 劉[UNK]即位，是為漢章帝
final prediction 明
find [UNK] in prediction 免再次

...

find [UNK] in prediction 李[UNK]
final prediction 農
find [UNK] in prediction 朱允[UNK]
final prediction 昌
find [UNK] in prediction [UNK]崎八幡宮
final prediction 內
find [UNK] in prediction 與慕容[UNK]雙方不和
final prediction 渾
find [UNK] in prediction 朱允[UNK]的禁殺之旨
final prediction 朱棣率領十餘人在盛庸營地附近露宿；次日清晨，發現被中央軍包圍。朱棣再次利用禁殺之旨，引馬鳴角，穿過敵軍，揚長而去。中央軍愕然，不敢射箭。朱棣回到營中，鼓勵眾將「兩軍相當，將勇者勝」，於是燕軍在東北方向，盛庸軍在西南方向，再次會戰。燕軍左右衝擊，盛庸堅守，雙方互有勝負。戰鬥打了三四個時辰後，突然颳起了強烈的東北風，塵埃蔽天。中央軍頂著風沙，根本沒法作戰；燕軍則乘風大呼，縱左右翼橫擊之，盛庸大敗，損失了數萬人後，退回德州。吳傑、平安引兵準備會合盛庸，聞庸已敗，退回真定。夾河之戰結束。夾河之戰重新確立了燕軍的優勢。閏三月初四，朱允炆因夾河之敗，再次罷免齊泰、黃子澄，謫出京城，暗中令其募兵。擊敗盛庸後，朱棣進軍真定。考慮到攻城較困難，決定誘敵出戰。於是下令軍中四出取糧，而用間諜向吳傑等透露此事。吳傑等見狀，決定襲擊之。閏三月初九，兩軍會於桑城，遂交戰。中央軍列方陣，朱棣則率精銳攻其東北角以破陣。燕將薛祿由於馬失足被擒，但奪敵刀斬數人後，奪馬逃走。此戰大量燕軍被中央軍的火槍和弓弩所傷，朱棣的帥旗被射得像刺蝟一樣；但是，由於朱允炆的禁殺之旨，朱棣本人反而沒事。
find [UNK] in prediction 杜恆-[UNK]因論題
final prediction 實
find [UNK] in prediction 青翁三足[UNK]
final prediction 博
find [UNK] in prediction [UNK][UNK]
final prediction 
find [UNK] in prediction 朱允[UNK]
final prediction 於
find [UNK] in prediction 免再次爆發內[U

  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 5 | Step 200 | loss = 0.003, acc = 0.937
Epoch 5 | Step 400 | loss = 0.003, acc = 0.944
Epoch 5 | Step 600 | loss = 0.004, acc = 0.929
Epoch 5 | Step 800 | loss = 0.005, acc = 0.926
Epoch 5 | Step 1000 | loss = 0.005, acc = 0.935
Epoch 5 | Step 1200 | loss = 0.005, acc = 0.921
Epoch 5 | Step 1400 | loss = 0.004, acc = 0.934
Epoch 5 | Step 1600 | loss = 0.005, acc = 0.920
Epoch 5 | Step 1800 | loss = 0.005, acc = 0.933
Epoch 5 | Step 2000 | loss = 0.005, acc = 0.922
Epoch 5 | Step 2200 | loss = 0.005, acc = 0.921
Epoch 5 | Step 2400 | loss = 0.005, acc = 0.917
Epoch 5 | Step 2600 | loss = 0.005, acc = 0.916
Epoch 5 | Step 2800 | loss = 0.005, acc = 0.921
Epoch 5 | Step 3000 | loss = 0.005, acc = 0.910
Epoch 5 | Step 3200 | loss = 0.005, acc = 0.913
Epoch 5 | Step 3400 | loss = 0.005, acc = 0.920
Epoch 5 | Step 3600 | loss = 0.005, acc = 0.921
Epoch 5 | Step 3800 | loss = 0.005, acc = 0.918
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

find [UNK] in prediction 李[UNK]
final prediction 農
find [UNK] in prediction 朱允[UNK]
final prediction 昌
find [UNK] in prediction [UNK]崎八幡宮
final prediction 內
find [UNK] in prediction [UNK]神星
final prediction 王
find [UNK] in prediction 與慕容[UNK]雙方不和
final prediction 渾
find [UNK] in prediction 朱允[UNK]的禁殺之旨
final prediction 朱棣率領十餘人在盛庸營地附近露宿；次日清晨，發現被中央軍包圍。朱棣再次利用禁殺之旨，引馬鳴角，穿過敵軍，揚長而去。中央軍愕然，不敢射箭。朱棣回到營中，鼓勵眾將「兩軍相當，將勇者勝」，於是燕軍在東北方向，盛庸軍在西南方向，再次會戰。燕軍左右衝擊，盛庸堅守，雙方互有勝負。戰鬥打了三四個時辰後，突然颳起了強烈的東北風，塵埃蔽天。中央軍頂著風沙，根本沒法作戰；燕軍則乘風大呼，縱左右翼橫擊之，盛庸大敗，損失了數萬人後，退回德州。吳傑、平安引兵準備會合盛庸，聞庸已敗，退回真定。夾河之戰結束。夾河之戰重新確立了燕軍的優勢。閏三月初四，朱允炆因夾河之敗，再次罷免齊泰、黃子澄，謫出京城，暗中令其募兵。擊敗盛庸後，朱棣進軍真定。考慮到攻城較困難，決定誘敵出戰。於是下令軍中四出取糧，而用間諜向吳傑等透露此事。吳傑等見狀，決定襲擊之。閏三月初九，兩軍會於桑城，遂交戰。中央軍列方陣，朱棣則率精銳攻其東北角以破陣。燕將薛祿由於馬失足被擒，但奪敵刀斬數人後，奪馬逃走。此戰大量燕軍被中央軍的火槍和弓弩所傷，朱棣的帥旗被射得像刺蝟一樣；但是，由於朱允炆的禁殺之旨，朱棣本人反而沒事。
find [UNK] in prediction 2[UNK]0
final prediction 有
find [UNK] in prediction 杜恆-[UNK]因論題
final prediction 實
find [UNK] in prediction 無錫胡[UNK]
final prediction 年
find [UNK] in prediction [UNK][UNK]


## Testing

In [11]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result10.csv"
with open(result_file, 'w',encoding="utf-8") as f:	
	f.write("ID,Answer\n")
	for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

find [UNK] in prediction 溥[UNK]
final prediction 
find [UNK] in prediction 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
final prediction 1
find [UNK] in prediction [UNK]人國
final prediction 1
find [UNK] in prediction 朱允[UNK]
final prediction 
find [UNK] in prediction 馬[UNK]
final prediction 1
find [UNK] in prediction 東晉常[UNK]
final prediction 
find [UNK] in prediction [UNK][UNK]
final prediction 1
find [UNK] in prediction [UNK]稻
final prediction 1
find [UNK] in prediction 白[UNK]紀滅絕事件
final prediction 1
find [UNK] in prediction 抗佝[UNK]病
final prediction 1
find [UNK] in prediction 杭州[UNK]橋機場
final prediction 
find [UNK] in prediction 蔡[UNK]
final prediction 1
find [UNK] in prediction 丁[UNK]
final prediction 1
find [UNK] in prediction 隋[UNK]帝
final prediction 1
find [UNK] in prediction 胡季[UNK]
final prediction 
find [UNK] in prediction 其英文縮寫首字母為「[UNK]·ㄎㄟ·ㄨㄞ」
final prediction 1
find [UNK] in prediction 梁[UNK]
final prediction 
find [UNK] in prediction [UNK]靼海峽
final prediction 1
find [UNK] in prediction 白[U